In [0]:
titles_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("/Volumes/workspace/default/netflix/titles.csv") 
titles_df.write.format("delta").mode("overwrite").saveAsTable("titles_df")
titles_df = spark.read.table("titles_df")

In [0]:
credits_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("/Volumes/workspace/default/netflix/credits.csv") 
credits_df.write.format("delta").mode("overwrite").saveAsTable("credits_df")

In [0]:
credits_df = spark.read.table("credits_df")

In [0]:
print(titles_df.dtypes)
display(titles_df.limit(10))

[('id', 'string'), ('title', 'string'), ('type', 'string'), ('description', 'string'), ('release_year', 'string'), ('age_certification', 'string'), ('runtime', 'string'), ('genres', 'string'), ('production_countries', 'string'), ('seasons', 'string'), ('imdb_id', 'string'), ('imdb_score', 'string'), ('imdb_votes', 'string'), ('tmdb_popularity', 'string'), ('tmdb_score', 'string')]


id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,1976,R,114,"['drama', 'crime']",['US'],null,tt0075314,8.2,808582.0,40.965,8.179
tm154986,Deliverance,MOVIE,"Intent on seeing the Cahulawassee River before it's turned into one huge lake, outdoor fanatic Lewis Medlock takes his friends on a river-rafting trip they'll never forget into the dangerous American back-country.",1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],null,tt0068473,7.7,107673.0,10.01,7.3
tm127384,Monty Python and the Holy Grail,MOVIE,"""King Arthur, accompanied by his squire, recruits his Knights of the Round Table, including Sir Bedevere the Wise, Sir Lancelot the Brave, Sir Robin the Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the Pure. On the way, Arthur battles the Black Knight who, despite having had all his limbs chopped off, insists he can still fight. They reach Camelot, but Arthur decides not to enter, as """"it is a silly place"""".""",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],null,tt0071853,8.2,534486.0,15.461,7.811
tm120801,The Dirty Dozen,MOVIE,"12 American military prisoners in World War II are ordered to infiltrate a well-guarded enemy château and kill the Nazi officers vacationing there. The soldiers, most of whom are facing death sentences for a variety of violent crimes, agree to the mission and the possible commuting of their sentences.",1967,TV-MA,150,"['war', 'action']","['GB', 'US']",null,tt0061578,7.7,72662.0,20.398,7.6
ts22164,Monty Python's Flying Circus,SHOW,"A British sketch comedy series with the shows being composed of surreality, risqué or innuendo-laden humour, sight gags and observational sketches without punchlines.",1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306
tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, but through a series of ridiculous events, he gains a reputation as the Messiah. When he's not dodging his followers or being scolded by his shrill mother, the hapless Brian has to contend with the pompous Pontius Pilate and acronym-obsessed members of a separatist movement. Rife with Monty Python's signature absurdity, the tale finds Brian's life paralleling Biblical lore, albeit with many more laughs.",1979,R,94,['comedy'],['GB'],null,tt0079470,8.0,395024.0,17.77,7.8
tm14873,Dirty Harry,MOVIE,"When a madman dubbed 'Scorpio' terrorizes San Francisco, hard-nosed cop, Harry Callahan – famous for his take-no-prisoners approach to law enforcement – is tasked with hunting down the psychopath. Harry eventually collars Scorpio in the process of rescuing a kidnap victim, only to see him walk on technicalities. Now, the maverick detective is determined to nail the maniac himself.",1971,R,102,"['thriller', 'action', 'crime']",['US'],null,tt0066999,7.7,155051.0,12.817,7.5
tm119281,Bonnie and Clyde,MOVIE,"In the 1930s, bored waitress Bonnie Parker falls in love with an ex-con named Clyde Barrow and together they start a violent crime spree through the country, stealing cars and robbing banks.",1967,R,110,"['crime', 'drama', 'action']",['US'],null,tt0061418,7.7,112048.0,15.687,7.5
tm98978,The Blue Lagoon,MOVIE,"Two small children and a ship's cook survive a shipwreck and find safety on an idyllic tropical island. Soon, however, the cook dies and the young boy and girl are left on their own. Days become years and Emmeline and Richard make a home for themselves surrounded by exotic creatures and nature's beauty. But will they ever see civilization again?",1980,R,104,"['romance', 'action', 'drama']",['US'],null,tt0080453,5.8,69844.0,50.324,6.156
tm44204,The Guns of Navarone,MOVIE,"A team of allied saboteurs are assigned an impossible mission: infil

In [0]:
from pyspark.sql.functions import col, isnull, count, when
nulls = titles_df.select([count(when(isnull(c),c)).alias(c) for c in titles_df.columns])
nulls.display()

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3,18,40,70,2594,121,97,96,3684,541,575,600,223,418


In [0]:
critical_col = ['id', 'title', 'release_year', 'imdb_id']
titles_df = titles_df.dropna(subset=critical_col)

In [0]:
titles_df = titles_df.fillna("No Description", "description")

In [0]:
import random
ty = ["MOVIE","SHOW"]
titles_df = titles_df.fillna(random.choice(ty),"type")

In [0]:
from pyspark.sql.functions import desc
mode = (titles_df.filter(col("age_certification").isNotNull()).groupBy("age_certification").count().orderBy(desc("count")).first()["age_certification"])
print(mode)
titles_df = titles_df.fillna(mode,"age_certification")

TV-MA


In [0]:
# from pyspark.sql.functions import mean
# mean_runtime = titles_df.select(mean("runtime")).first()[0]
# print(mean_runtime)
# titles_df = titles_df.fillna(mean_runtime, "runtime")


In [0]:
joined_df = titles_df.join(credits_df, on="id", how="inner")


In [0]:
joined_df.show()

+-------+-----------+-----+--------------------+------------+-----------------+-------+------------------+--------------------+-------+---------+----------+----------+---------------+----------+---------+------------------+--------------------+-----+
|     id|      title| type|         description|release_year|age_certification|runtime|            genres|production_countries|seasons|  imdb_id|imdb_score|imdb_votes|tmdb_popularity|tmdb_score|person_id|              name|           character| role|
+-------+-----------+-----+--------------------+------------+-----------------+-------+------------------+--------------------+-------+---------+----------+----------+---------------+----------+---------+------------------+--------------------+-----+
|tm84618|Taxi Driver|MOVIE|A mentally unstab...|        1976|                R|    114|['drama', 'crime']|              ['US']|   NULL|tt0075314|       8.2|  808582.0|         40.965|     8.179|     3748|    Robert De Niro|       Travis Bickle|ACT

In [0]:
credits_df.groupBy("name") \
    .count().orderBy("count", ascending=False).limit(10).show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|         Boman Irani|   25|
| Kareena Kapoor Khan|   25|
|      Shah Rukh Khan|   23|
|    Takahiro Sakurai|   21|
|         Raúl Campos|   20|
|        Paresh Rawal|   20|
|    Amitabh Bachchan|   20|
|Priyanka Chopra J...|   20|
|           Jan Suter|   19|
|          Aamir Khan|   19|
+--------------------+-----+



In [0]:
titles_df.groupBy("type").count().show()

+--------------------+-----+
|                type|count|
+--------------------+-----+
| which broke the ...|    1|
| west against the...|    1|
| and their improv...|    1|
|               TV-14|    5|
|                NULL|   18|
| she witnesses by...|    1|
| which is made by...|    1|
|   including Spanish|    1|
| a children's str...|    1|
| one thing is cle...|    1|
| and was eventual...|    1|
| classmates and t...|    1|
|                2019|    1|
|                2017|    1|
| cast out of the ...|    1|
| which is driving...|    1|
|              ['US']|    1|
|               MOVIE| 3744|
|         the Octopod|    1|
|          two courts|    1|
+--------------------+-----+
only showing top 20 rows


In [0]:
titles_df.filter("type = 'MOVIE' and imdb_score is not null") \
    .orderBy("imdb_score", ascending=False).select("title", "imdb_score").limit(10).show()

+--------------------+----------+
|               title|imdb_score|
+--------------------+----------+
|ReMastered: The L...| tt9046576|
|        Chaman Bahar| tt8747450|
|Dance Dreams: Hot...| tt8741182|
|Derren Brown: Mir...| tt8599562|
|W. Kamau Bell: Pr...| tt8467788|
|D.L. Hughley: Con...| tt8426270|
|John Mulaney: Kid...| tt8271714|
|No One Will Ever ...| tt8251336|
|Agustín Aristarán...| tt8004552|
|Ronnie Coleman: T...| tt7217214|
+--------------------+----------+



In [0]:
titles_df.groupBy("release_year").count().orderBy("release_year").show()

+--------------------+-----+
|        release_year|count|
+--------------------+-----+
|                NULL|   70|
| ""Married..."" f...|    1|
| ""Sound of your ...|    1|
| 2010 after the V...|    1|
| Abdus Salam. The...|    1|
| Autumn's Concert...|    1|
| Barbie’s precoci...|    1|
| Bert Kreischer: ...|    1|
| Birbiglia tiptoe...|    1|
| Chaldea issues a...|    1|
|              Cheryl|    1|
| David visits the...|    1|
| Dennō Coil was h...|    1|
|             Italian|    1|
| James. Soon the ...|    1|
| Jim's obsession ...|    1|
| Jr. and Zen Mast...|    1|
|              Julian|    1|
| Kagome locates a...|    1|
|              Lisbon|    1|
+--------------------+-----+
only showing top 20 rows


In [0]:
credits_df.filter("role = 'ACTOR'").groupBy("name") \
    .count().orderBy("count", ascending=False).limit(10).show()


+--------------------+-----+
|                name|count|
+--------------------+-----+
| Kareena Kapoor Khan|   25|
|         Boman Irani|   25|
|      Shah Rukh Khan|   23|
|    Takahiro Sakurai|   21|
|Priyanka Chopra J...|   20|
|        Paresh Rawal|   20|
|    Amitabh Bachchan|   20|
| Nawazuddin Siddiqui|   19|
|         Anupam Kher|   19|
|      Junichi Suwabe|   19|
+--------------------+-----+



In [0]:
credits_df.filter("name = 'Robert De Niro'") \
    .join(titles_df, "id").select("title", "release_year", "imdb_score").show()

+--------------------+--------------------+----------+
|               title|        release_year|imdb_score|
+--------------------+--------------------+----------+
|         Taxi Driver|                1976|       8.2|
|          GoodFellas|                1990|       8.7|
|Once Upon a Time ...|                1984|       8.3|
|          Awakenings| filled with awe ...|    ['US']|
|Jim Norton: Mouth...|                2017|       7.0|
|        The Irishman|                2019|       7.8|
|The Irishman: In ...|                2019|      NULL|
+--------------------+--------------------+----------+



In [0]:
titles_df.withColumn("genre", explode("genres")) \
    .groupBy("genre").avg("tmdb_score").orderBy("avg(tmdb_score)", ascending=False).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6567264660263619>, line 2
      1 titles_df.withColumn("genre", explode("genres")) \
----> 2     .groupBy("genre").avg("tmdb_score").orderBy("avg(tmdb_score)", ascending=False).show()

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:536, in DataFrame.groupBy(self, *cols)
    534     _cols.append(c)
    535 elif isinstance(c, str):
--> 536     _cols.append(self[c])
    537 elif isinstance(c, int) and not isinstance(c, bool):
    538     if c < 1:

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:1860, in DataFrame.__getitem__(self, item)
   1856     from pyspark.sql.connect.types import verify_col_name
   1858     # Try best to verify the column name with cached schema
   1859     # If fails, fall back to the server side validation
-> 1860     if not

In [0]:
titles_df.filter("type = 'SHOW'") \
    .orderBy("tmdb_popularity", ascending=False).select("title", "tmdb_popularity").show()

+--------------------+--------------------+
|               title|     tmdb_popularity|
+--------------------+--------------------+
|            The Mire|           tt8855592|
|        Dark Tourist|           tt8725166|
|Back Street Girls...|           tt8693818|
|   Sirius the Jaeger|           tt8565136|
|The Sound of Your...|           tt7885558|
|     Wedding Planner|           tt7885290|
|                Ajin|           tt5537534|
|Transformers: Rob...|           tt3604232|
|           Swap Shop|          tt15674202|
| The Uncanny Counter|          tt13273826|
|           Valentino|          tt11508212|
|       Darwin's Game|          tt11405390|
|          Dorohedoro|          tt11147852|
|       The Blacklist|['thriller', 'dra...|
|            The Mole|         ['reality']|
| Merli: Dare to Know| ['comedy', 'drama']|
|        Robot Trains|['animation', 'ac...|
|           One Piece|['animation', 'ac...|
|Married at First ...|              ['US']|
|       นางสาวก้นครัว|          